In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
sys.path.append('..')
import calibration

In [61]:
def pipeline_calibration_noinstance(a1elist,a2elist,aelist=None,iters_perfect=1000,iters_zero=1000):
    '''
    calibration_pipeline_noinstance(a1elist,a2elist,aelist=None,iters_perfect=1000,iters_zero=1000)
    
    Calibration pipeline without generating instances. The pos and neg are calculated from P matrix.
    
    Parameters
    ----------
    aelist: The poll for randomizing a1 and a2. When aelist=None, aelist will be the union of a1elist and a2elist.
    
    Returns
    -------
    obs,neg1_mean,neg1_sigma,neg2_mean,neg2_sigma,pos1_mean,pos1_sigma,pos2_mean,pos2_sigma,pos_mean,pos_sigma,s1,s1_sigma
    '''
    
    ## convert to neighborhood
    a1dict = calibration.formatting.edgelist_to_neighborhood(a1elist)
    a2dict = calibration.formatting.edgelist_to_neighborhood(a2elist)

    ## observed overlap
    obs = calibration.count_overlap(a1elist,a2elist)

    ## positive benchmark
    if aelist==None:
        aelist = list(set(a1elist).union(a2elist)) # union of a1 and a2
    adict = calibration.formatting.edgelist_to_neighborhood(aelist) 
    alphas1,alpha_probe1 = calibration.random_subnetwork.optimize_alpha(adict,a1dict,iters=iters_perfect,probeNode=0)
    alphas2,alpha_probe2 = calibration.random_subnetwork.optimize_alpha(adict,a2dict,iters=iters_perfect,probeNode=0)
    # average probability
    P1 = calibration.random_subnetwork.cal_probability(aelist,a1elist,alphas=alphas1)
    P2 = calibration.random_subnetwork.cal_probability(aelist,a2elist,alphas=alphas2)
    # variance probability
    Pv1 = {link: P1[link]*(1-P1[link]) for link in P1 }
    Pv2 = {link: P2[link]*(1-P2[link]) for link in P2 }
    # benchmarks
        # one side
    pos1_mean = sum([P1.get(link,0) for link in a2elist])
    pos1_sigma = np.sqrt(sum([Pv1.get(link,0) for link in a2elist]))
    pos2_mean = sum([P2.get(link,0) for link in a1elist])
    pos2_sigma = np.sqrt(sum([Pv2.get(link,0) for link in a1elist]))
        # two side
    comLinks = set(P1.keys()).intersection(P2.keys())
    P = {comLink:P1[comLink]*P2[comLink] for comLink in comLinks}
    Pv = {link: P[link]*(1-P[link]) for link in P }
    pos_mean = sum(P.values())
    pos_sigma = np.sqrt(sum(Pv.values()))

    ## negative benchmark
    # neg1
    a1nodes = list(sorted(a1dict.keys()))
    alphas1_neg = calibration.random_network.optimize_alpha(a1dict,iters=iters_zero)
    # only calculate the links in the comparing network(a2elist). Other links will never overlap with the comparing network.
    P1 = calibration.random_subnetwork.cal_probability(a2elist,a1elist,alphas1_neg) 
    Pv1 = {link: P1[link]*(1-P1[link]) for link in P1 }
    # one side
    neg1_mean = sum([P1.get(link,0) for link in a2elist])
    neg1_sigma = np.sqrt(sum([Pv1.get(link,0) for link in a2elist]))

    # neg2
    a2nodes = list(sorted(a2dict.keys()))
    alphas2_neg = calibration.random_network.optimize_alpha(a2dict,iters=iters_zero)
    # only calculate the links in the comparing network(a1elist). Other links will never overlap with the comparing network.
    P2 = calibration.random_subnetwork.cal_probability(a1elist,a2elist,alphas2_neg) 
    Pv2 = {link: P2[link]*(1-P2[link]) for link in P2 }
    # one side
    neg2_mean = sum([P2.get(link,0) for link in a1elist])
    neg2_sigma = np.sqrt(sum([Pv2.get(link,0) for link in a1elist]))
    
    return obs,neg1_mean,neg1_sigma,neg2_mean,neg2_sigma,pos1_mean,pos1_sigma,pos2_mean,pos2_sigma,pos_mean,pos_sigma

In [2]:
df_abbi = pd.read_csv("../datasets/yeast/ito_uetz_ccsb_yeri/ABBI-21.tsv",sep = "\t").drop(columns=["gene_name_a","gene_name_b","Zone","GI_PCC","CS_PCC","GE_PCC"])
df_ito = df_abbi[df_abbi["Ito-core"]==True].iloc[:,:2]
df_uetz = df_abbi[df_abbi["Uetz-screen"]==True].iloc[:,:2]
df_ccsb = df_abbi[df_abbi["CCSB-YI1"]==True].iloc[:,:2]
df_yeri = df_abbi[df_abbi["YeRI"]==True].iloc[:,:2]

In [3]:
yeriElist = calibration.sort_elist(df_yeri.values)
itoElist = calibration.sort_elist(df_ito.values)

In [64]:
%%time
pipeline_calibration_noinstance(yeriElist,itoElist)

CPU times: user 14.7 s, sys: 66.1 ms, total: 14.7 s
Wall time: 14.8 s


(73,
 2.0713781330849055,
 1.392021131374764,
 3.158114585129418,
 1.710183625110492,
 77.19438033798285,
 2.4036443824735527,
 75.87345594999958,
 2.2424599318224683,
 79.03175445204654,
 3.1989524210424034)

In [6]:
from Pipeline import Pipeline
pipe = Pipeline(yeriElist,itoElist)
pipe.show_results(printOn=False)

(['Observed overlap',
  'Neg_mean',
  'Neg_sigma',
  'Neg_p',
  'Pos_mean',
  'Pos_sigma',
  'Pos_p',
  'Normlap',
  'Normlap_sigma'],
 [73,
  3.1581145851294155,
  1.7101936251104919,
  0.0,
  75.87345594999962,
  2.2424699318224683,
  0.10003012618139162,
  0.9604834977590048,
  0.030853022130100192])

In [53]:
elist1 = yeriElist
elist2 = itoElist

# transfer to ids
id2name = {}
name2id = {}

nodes1 = set([node for pair in elist1 for node in pair])
nodes2 = set([node for pair in elist2 for node in pair])
nodes = sorted(nodes1.union(nodes2))

for i, node in enumerate(nodes):
    id2name[i] = node
    name2id[node] = i

elist1 = [(name2id[node1],name2id[node2]) for node1,node2 in elist1]
elist2 = [(name2id[node1],name2id[node2]) for node1,node2 in elist2]


In [60]:
%%time
calibration.count_overlap(elist1,elist2)

CPU times: user 153 µs, sys: 12 µs, total: 165 µs
Wall time: 168 µs


73